#### Carregando bibliotecas necessárias

In [1]:
import pandas as pd 
import numpy as np
import requests
from dotenv import load_dotenv
import os
import json
from datetime import datetime, timedelta
from google.cloud import bigquery
import pandas_gbq

#### Carregando variaveis de ambiente

In [2]:
load_dotenv()

True

#### utilizando as váriaveis de ambiente

In [3]:
api_key = os.getenv("COINCAP_API_KEY")
google_credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

#### Realizando a primeira requisição e carregando pra camada bronze do projeto

In [4]:
def extract_dados(api_key, endpoint):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    url = f"https://api.coincap.io/v2/{endpoint}"
    try: 
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        dados = response.json().get("data", [])
        return dados 
    except requests.exceptions.RequestsDependencyWarning as e: 
        print(f"Erro ao acessos {endpoint}: {e}")
        return []

#### Criando o dataframe assets

In [5]:
if __name__ == "__main__":
    dados_assets = extract_dados(api_key, "assets") #Se precisar fazer requisição em outro endpoint, só trocar o "assets"
    bronze_assets = pd.DataFrame(dados_assets)
    bronze_assets["extract_date"] = pd.Timestamp.now()

### Criando o dataframe markets

In [6]:
if __name__ == "__main__":
    dados_markets = extract_dados(api_key, "markets")
    bronze_markets = pd.DataFrame(dados_markets)
    bronze_markets["extract_date"] = pd.Timestamp.now()

#### Checando os dataframes

In [7]:
with pd.option_context("display.max_columns", None):
    display(bronze_assets.head(5))

,id,rank,symbol,name,supply,maxSupply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr,vwap24Hr,explorer,extract_date
0,bitcoin,1,BTC,Bitcoin,19812484.0000000000000000,21000000.0000000000000000,2020295263479.2918947762287800,21624782212.0350528212145144,101970.8211993656067950,-3.0296931016556670,103831.6976744818501080,https://blockchain.info/,2025-01-19 23:52:44.156804
1,ethereum,2,ETH,Ethereum,120501725.2119089500000000,None,392527166346.0541520484023715,20083397171.8580631130072172,3257.4402205094857790,-3.1550717782806308,3288.7018609687066522,https://etherscan.io/,2025-01-19 23:52:44.156804
2,xrp,3,XRP,XRP,57564441898.0000000000000000,100000000000.0000000000000000,177472950555.5378050320497106,5215551570.2663705863777963,3.0830308555758597,-5.6126007823319695,3.1138006022755506,https://xrpcharts.ripple.com/#/graph/,2025-01-19 23:52:44.156804
3,tether,4,USDT,Tether,138368309376.0522200000000000,None,138277892160.9029727648955158,78207492934.1483687536043136,0.9993465467955996,-0.0029761703696144,0.9988293772901204,https://www.omniexplorer.info/asset/31,2025-01-19 23:52:44.156804
4,solana,5,SOL,Solana,486598199.3482777000000000,None,114771112565.5574025973794271,10770368406.8259258336248644,235.8642360766550000,-7.9964286627502536,270.5878179103873619,https://explorer.solana.com/,2025-01-19 23:52:44.156804


In [52]:
with pd.option_context("display.max_columns", None):
    display(bronze_markets.head(5))

,exchangeId,rank,baseSymbol,baseId,quoteSymbol,quoteId,priceQuote,priceUsd,volumeUsd24Hr,percentExchangeVolume,tradesCount24Hr,updated,extract_date
0,bibox,1,ETH,ethereum,USDT,tether,3313.7600000000000000,3310.7113408000000000,31082173.7606347417600000,15.7069144508125571,None,1737248277239,2025-01-18 22:01:41.548605
1,bibox,2,LINK,chainlink,USDT,tether,23.8740000000000000,23.8520359200000000,13132789.9881676768800000,6.6364601920436892,None,1737248416642,2025-01-18 22:01:41.548605
2,bibox,3,ATOM,cosmos,USDT,tether,6.6644000000000000,6.6582687520000000,7618772.6248378896000000,3.8500334873643531,None,1737248395462,2025-01-18 22:01:41.548605
3,bibox,4,XLM,stellar,USDT,tether,0.4903760000000000,0.4899248540800000,6432733.5219139372204800,3.2506862580358031,None,1737247801768,2025-01-18 22:01:41.548605
4,bibox,5,FIL,filecoin,USDT,tether,5.4800000000000000,5.4749584000000000,5910920.0814273289600000,2.9869955930222906,None,1737245994746,2025-01-18 22:01:41.548605


### Criando a camada bronze no GCP.

#### Caso precise criar o Dataset de maneira prévia no seu projeto

In [6]:
#def create_bigquery_dataset(project_id: str, dataset_name: str, location: str = "US"):
#    try:
#        client = bigquery.Client(project=project_id)
#        dataset_id = f"{project_id}.{dataset_name}"
#        dataset = bigquery.Dataset(dataset_id)
#        dataset.location = location
#        dataset = client.create_dataset(dataset, exists_ok=True)
#        print(f"Conjunto de dados '{dataset_id}' criado com sucesso na localização '{location}'.")
#    except Exception as e:
#        print(f"Erro ao criar o conjunto de dados: {e}")

#### Criando o conjunto de dados no projeto dentro do GCP

In [ ]:
#if __name__ == "__main__":
#    create_bigquery_dataset("mystudentproject-292517", "Engenharia_Dados_Cadastra")

Conjunto de dados 'mystudentproject-292517.Engenharia_Dados_Cadastra' criado com sucesso na localização 'US'.


#### Utilizando pandas_gbq para criar o dataset e tabelas na camada bronze.

In [ ]:
#export_to_gbq_with_condition(bronze_assets, 'cadastra_teste_engenharia.bronze_assets', 'mystudentproject-292517')

In [53]:
pandas_gbq.to_gbq(bronze_assets, 'cadastra_teste_engenharia.bronze_assets', project_id='mystudentproject-292517', if_exists='append')

100%|██████████| 1/1 [00:00<?, ?it/s]


In [54]:
pandas_gbq.to_gbq(bronze_markets, 'cadastra_teste_engenharia.bronze_markets', project_id='mystudentproject-292517', if_exists='append')

100%|██████████| 1/1 [00:00<?, ?it/s]


#### Criação da camada Silver, utilizando as tabelas no BigQuery.

In [68]:
def bronze_layer_extract(query):
    try:
        client = bigquery.Client()
        result = client.query(query).to_dataframe()
        return result
    except Exception as e: 
        print(f"Erro ao executar consulta: {e}")
        return None

In [110]:
if __name__ == "__main__":
    client = bigquery.Client()
    query = """
    SELECT * FROM `mystudentproject-292517.cadastra_teste_engenharia.bronze_assets`;
    """
    silver_assets = bronze_layer_extract(query)

In [111]:
if __name__ == "__main__":
    client = bigquery.Client()
    query = """
    SELECT * FROM `mystudentproject-292517.cadastra_teste_engenharia.bronze_markets`;
    """
    silver_markets = bronze_layer_extract(query)

#### Tratamento necessários nesses dataframes.

In [112]:
def clean_null_values(df, fillna_map):
    for column, fill_value in fillna_map.items():
        if column in df.columns: 
            df[column] = df[column].fillna(fill_value)
    return df

In [113]:
def convert_to_numeric(df, numeric_columns):
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
    return df

In [114]:
def process_dataframe(df, fillna_map, numeric_columns):
    # Limpar valores nulos
    df = clean_null_values(df, fillna_map)
    # Converter colunas numéricas
    df = convert_to_numeric(df, numeric_columns)
    return df

#### Encontra valores nulos

In [119]:
def find_null_or_empty_columns(df):
    null_columns = []
    for column in df.columns:
        if df[column].isnull().any() or (df[column] == "").any():
            null_columns.append(column)
    return null_columns

In [120]:
colunas_nulas = find_null_or_empty_columns(silver_assets)
print("Colunas com valores nulos ou vazios:", colunas_nulas)

Colunas com valores nulos ou vazios: ['maxSupply', 'changePercent24Hr', 'explorer']


In [121]:
colunas_nulas = find_null_or_empty_columns(silver_markets)
print("Colunas com valores nulos ou vazios:", colunas_nulas)

Colunas com valores nulos ou vazios: ['tradesCount24Hr']


In [122]:
silver_markets['tradesCount24Hr'].drop_duplicates()

0    None
Name: tradesCount24Hr, dtype: object

In [123]:
silver_markets = silver_markets.drop(columns=["tradesCount24Hr"])

In [97]:
fillna_map = {
    "maxSupply":0,
    'changePercent24Hr':0,
    'explorer': "link_indisponivel"
}

In [98]:
numeric_columns = ['rank', 'supply', 'maxSupply', 'marketCapUsd', 'volumeUsd24Hr', 'priceUsd', 'changePercent24Hr', 'vwap24Hr']

In [125]:
silver_assets = process_dataframe(silver_assets, fillna_map, numeric_columns)

In [129]:
numeric_columns = ["priceQuote","priceUsd","volumeUsd24Hr","percentExchangeVolume"]

In [131]:
silver_markets = process_dataframe(silver_markets, fillna_map, numeric_columns)

In [133]:
silver_markets["updated_date"] = pd.to_datetime(silver_markets['updated'], unit='ms')

### Localizando colunas com valores nulos

In [126]:
with pd.option_context("display.max_columns", None):
    display(silver_assets.tail())

,id,rank,symbol,name,supply,maxSupply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr,vwap24Hr,explorer,extract_date
95,vechain,29,VET,VeChain,8.098504e+10,8.671263e+10,4.239211e+09,5.952220e+07,0.052346,-5.888639,0.052372,https://explore.veforge.com/,2025-01-18 22:01:39.441831
96,xrp,3,XRP,XRP,5.756444e+10,1.000000e+11,1.880043e+11,3.558535e+09,3.265980,0.339867,3.182373,https://xrpcharts.ripple.com/#/graph/,2025-01-18 22:01:39.441831
97,reserve-rights,72,RSR,Reserve Rights,5.383409e+10,1.000000e+11,8.088785e+08,5.553037e+07,0.015025,-7.336334,0.014817,https://etherscan.io/token/0x8762db106b2c2a0bc...,2025-01-18 22:01:39.441831
98,telcoin,84,TEL,Telcoin,9.187008e+10,1.000000e+11,5.904458e+08,1.922761e+06,0.006427,-0.711426,0.006230,https://etherscan.io/token/0x85e076361cc813a90...,2025-01-18 22:01:39.441831
99,ecash,77,XEC,eCash,1.981336e+13,2.100000e+13,7.179583e+08,6.183923e+06,0.000036,-6.836051,0.000037,https://explorer.bitcoinabc.org/,2025-01-18 22:01:39.441831


In [135]:
with pd.option_context("display.max_columns", None):
    display(silver_markets.head())

,exchangeId,rank,baseSymbol,baseId,quoteSymbol,quoteId,priceQuote,priceUsd,volumeUsd24Hr,percentExchangeVolume,updated,extract_date,updated_date
0,bibox,27,LTC,litecoin,ETH,ethereum,0.038108,126.151712,1.898533e+06,0.959396,1737248307122,2025-01-18 22:01:41.548605,2025-01-19 00:58:27.122
1,bibox,33,UMA,uma,ETH,ethereum,0.000810,2.681764,1.601311e+06,0.809199,1737242843422,2025-01-18 22:01:41.548605,2025-01-18 23:27:23.422
2,bibox,37,BSV,bitcoin-sv,ETH,ethereum,0.016829,55.710217,1.526764e+06,0.771527,1737248447769,2025-01-18 22:01:41.548605,2025-01-19 01:00:47.769
3,bibox,44,LINK,chainlink,ETH,ethereum,0.007206,23.852970,1.171840e+06,0.592172,1737248415392,2025-01-18 22:01:41.548605,2025-01-19 01:00:15.392
4,bibox,45,ETC,ethereum-classic,ETH,ethereum,0.008158,27.006925,1.073288e+06,0.542370,1737248288442,2025-01-18 22:01:41.548605,2025-01-19 00:58:08.442


### Criando as tabelas na camada Silver para receber os dados.

In [136]:
def create_bigquery_table(dataset_id: str, table_name: str, schema: list):
    try:
        client = bigquery.Client()
        table_id = f"{dataset_id}.{table_name}"
        table_schema = [bigquery.SchemaField(col["name"], col["type"], col.get("mode", "NULLABLE")) for col in schema]
        table = bigquery.Table(table_id, schema=table_schema)
        table = client.create_table(table)
        print(f"Tabela '{table_id}' criada com sucesso.")
    except Exception as e:
        print(f"Erro ao criar a tabela: {e}")

In [142]:
if __name__ == "__main__":
    schema = [
        {"name": "id", "type": "STRING"},
        {"name": "rank", "type": "STRING"},
        {"name": "symbol", "type": "STRING"},
        {"name": "name", "type": "STRING"},
        {"name": "supply", "type": "FLOAT"},
        {"name": "maxSupply", "type": "FLOAT"},
        {"name": "marketCapUsd", "type": "FLOAT"},
        {"name": "volumeUsd24Hr", "type": "FLOAT"},
        {"name": "priceUsd", "type": "FLOAT"},
        {"name": "changePercent24Hr", "type": "FLOAT"},
        {"name": "vwap24Hr", "type": "FLOAT"},
        {"name": "explorer", "type": "STRING"},
        {"name": "extract_date", "type": "DATE"}
    ]
    create_bigquery_table("mystudentproject-292517.cadastra_teste_engenharia", "silver_assets", schema)

Tabela 'mystudentproject-292517.cadastra_teste_engenharia.silver_assets' criada com sucesso.


In [143]:
pandas_gbq.to_gbq(silver_assets, 'cadastra_teste_engenharia.silver_assets', project_id='mystudentproject-292517', if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


-------------------

In [146]:
if __name__ == "__main__":
    schema = [
        {"name": "id", "type": "STRING"},
        {"name": "rank", "type": "STRING"},
        {"name": "symbol", "type": "STRING"},
        {"name": "name", "type": "STRING"},
        {"name": "supply", "type": "FLOAT"},
        {"name": "maxSupply", "type": "FLOAT"},
        {"name": "marketCapUsd", "type": "FLOAT"},
        {"name": "volumeUsd24Hr", "type": "FLOAT"},
        {"name": "priceUsd", "type": "FLOAT"},
        {"name": "changePercent24Hr", "type": "FLOAT"},
        {"name": "vwap24Hr", "type": "FLOAT"},
        {"name": "explorer", "type": "STRING"},
        {"name": "extract_date", "type": "DATE"},
        {"name": "updated_date", "type": "DATE"}
    ]
    create_bigquery_table("mystudentproject-292517.cadastra_teste_engenharia", "silver_markets", schema)

Tabela 'mystudentproject-292517.cadastra_teste_engenharia.silver_markets' criada com sucesso.


In [147]:
pandas_gbq.to_gbq(silver_markets, 'cadastra_teste_engenharia.silver_markets', project_id='mystudentproject-292517', if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]
